# Data pre-processing

In [1]:
import dlib
import cv2
import tensorflow as tf
import glob
import numpy as np
import matplotlib.image as mpimg
import os
nums = len(next(os.walk('Training_data'))[1])
print(len(next(os.walk('Training_data'))[1]))
path = 'Training_Data'
dir_list = os.listdir(path)

dir_list
label_list = []
for i in range(nums):
    label_list = label_list + [np.zeros(nums)]
    label_list[i][i] = 1
    
label_list

aa = [dir_list, label_list]

label_list

7


[array([1., 0., 0., 0., 0., 0., 0.]),
 array([0., 1., 0., 0., 0., 0., 0.]),
 array([0., 0., 1., 0., 0., 0., 0.]),
 array([0., 0., 0., 1., 0., 0., 0.]),
 array([0., 0., 0., 0., 1., 0., 0.]),
 array([0., 0., 0., 0., 0., 1., 0.]),
 array([0., 0., 0., 0., 0., 0., 1.])]

In [2]:
face_detector = dlib.get_frontal_face_detector()

shape_predictor = dlib.shape_predictor('Files/shape_predictor_68_face_landmarks.dat')

face_recognition_model = dlib.face_recognition_model_v1('Files/dlib_face_recognition_resnet_model_v1.dat')


images = []

label = []

        


def create_training_inputs(data_path, container, face_detector, shape_predictor, face_recognition_model,label):
    for i, file_name in enumerate(glob.glob(data_path)):
        aa = list(file_name.split("\\"))
        bb = dir_list.index(aa[1])
        im = mpimg.imread(file_name)
        if(file_name.endswith(".pgm")):
            im = np.stack((im,)*3, axis=-1)
        faces = face_detector(im, 1)
        shapes = [shape_predictor(im, face) for face in faces]
        if len(shapes) != 0:
            face_ids = [face_recognition_model.compute_face_descriptor(im, shape, 1) for shape in shapes]
            face_ids_array = np.array(face_ids)
            face_ids_list = face_ids_array.tolist()[0]
            container.append(face_ids_list)
            label.append(label_list[bb])
        else:
            print ("Cannot detect face in: ", file_name)

            
create_training_inputs("Training_Data/*/*", images, face_detector, shape_predictor, face_recognition_model,label)

Inputs = np.array(images)

label = np.array(label)

print ("Inputs shape: ", Inputs.shape)
print ("Label shape:", label.shape)

Cannot detect face in:  Training_Data\Runzhe\sb3 (2).jpg
Cannot detect face in:  Training_Data\Runzhe\sb3 (3).jpg
Cannot detect face in:  Training_Data\Runzhe\sb3 (4).jpg
Cannot detect face in:  Training_Data\Runzhe\sb3 (5).jpg
Cannot detect face in:  Training_Data\Runzhe\sb3 (6).jpg
Cannot detect face in:  Training_Data\Runzhe\sb3 (7).jpg
Cannot detect face in:  Training_Data\Runzhe\sb3 (8).jpg
Inputs shape:  (86, 128)
Label shape: (86, 7)


# Build Computational Graph - Tensorflow

In [3]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
# Hyperparameter
hidden_size1 = 300
hidden_size2 = 200
hidden_size3 = 100
hidden_size4 = 50




# Create placeholders
X  = tf.placeholder(tf.float32, [None, 128])
Y_ = tf.placeholder(tf.float32, [None, len(dir_list)])
lr = tf.placeholder(tf.float32)
pkeep = tf.placeholder(tf.float32)

# Create variables
w1 = tf.Variable(tf.truncated_normal([128, hidden_size1], stddev=0.1), tf.float32)
b1 = tf.Variable(tf.ones([hidden_size1])/10)

w2 = tf.Variable(tf.truncated_normal([hidden_size1, hidden_size2], stddev=0.1), tf.float32)
b2 = tf.Variable(tf.ones([hidden_size2])/10)

w3 = tf.Variable(tf.truncated_normal([hidden_size2, hidden_size3], stddev=0.1), tf.float32)
b3 = tf.Variable(tf.ones([hidden_size3])/10)

w4 = tf.Variable(tf.truncated_normal([hidden_size3, hidden_size4], stddev=0.1), tf.float32)
b4 = tf.Variable(tf.ones([hidden_size4])/10)

w5 = tf.Variable(tf.truncated_normal([hidden_size4, len(dir_list)], stddev=0.1), tf.float32)
b5 = tf.Variable(tf.ones([len(dir_list)])/10)


Y1 = tf.matmul(X, w1) + b1
Y1 = tf.nn.relu(Y1)

Y2 = tf.matmul(Y1, w2) + b2
Y2 = tf.nn.relu(Y2)

Y3 = tf.matmul(Y2, w3) + b3
Y3 = tf.nn.relu(Y3)

Y4 = tf.matmul(Y3, w4) + b4
Y4 = tf.nn.relu(Y4)



Ydrop = tf.nn.dropout(Y4, pkeep)

Ylogits = tf.matmul(Ydrop, w5) + b5
Y = tf.nn.softmax(Ylogits)

# Loss (Cross-entropy)
loss = tf.nn.softmax_cross_entropy_with_logits(logits=Ylogits, labels=Y_)

# Optimizer
optimizer = tf.train.AdamOptimizer(lr).minimize(loss)

# Accuracy
is_correct = tf.equal(tf.argmax(Y,1), tf.argmax(Y_,1))
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))

# Initializer
init = tf.global_variables_initializer()
saver = tf.train.Saver()

Instructions for updating:
non-resource variables are not supported in the long term
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



# Training Process

In [4]:
with tf.compat.v1.Session() as sess:
    sess.run(init)
    
    train_dict = {X: Inputs, Y_: label, lr: 0.0001, pkeep: 0.9}
    # Training
    for i in range(1000):
        sess.run(optimizer, feed_dict=train_dict)
        
        # Accuracy
        if i % 100 == 0:
            A, C = sess.run([accuracy, loss], feed_dict=train_dict)
            print ("At iteration: %s, Loss: %s, Accuracy: %s" % (i,sum(C),A))
    
    save_path = saver.save(sess, "Checkpoint/model.ckpt")
    print("Model saved in file: %s" % save_path)

At iteration: 0, Loss: 168.5761021375656, Accuracy: 0.10465116
At iteration: 100, Loss: 95.38324385881424, Accuracy: 0.6976744
At iteration: 200, Loss: 40.3262560851872, Accuracy: 0.8604651
At iteration: 300, Loss: 15.7063397183083, Accuracy: 0.95348835
At iteration: 400, Loss: 5.6949558245250955, Accuracy: 1.0
At iteration: 500, Loss: 1.638295668642968, Accuracy: 1.0
At iteration: 600, Loss: 0.7709358344436623, Accuracy: 1.0
At iteration: 700, Loss: 0.5891951007361058, Accuracy: 1.0
At iteration: 800, Loss: 0.5763916321739089, Accuracy: 1.0
At iteration: 900, Loss: 0.28727528136005276, Accuracy: 1.0
Type is unsupported, or the types of the items don't match field type in CollectionDef. Note this is a warning and probably safe to ignore.
tf.float32 has type DType, but expected one of: int, long, bool
Type is unsupported, or the types of the items don't match field type in CollectionDef. Note this is a warning and probably safe to ignore.
tf.float32 has type DType, but expected one of: 

In [5]:
import numpy as np
import dlib
import matplotlib.image as mpimg


face_detector = dlib.get_frontal_face_detector()

shape_predictor = dlib.shape_predictor('Files/shape_predictor_68_face_landmarks.dat')

face_recognition_model = dlib.face_recognition_model_v1('Files/dlib_face_recognition_resnet_model_v1.dat')


known_faces = []
for i in range(nums):
    path = "Training_Data/" + dir_list[i] + "/*"
    im = mpimg.imread(glob.glob(path)[0])
    if(glob.glob(path)[0].endswith(".pgm")):
        im = np.stack((im,)*3, axis=-1)
    faces = face_detector(im, 1)
    shapes = [shape_predictor(im, face) for face in faces]
    if len(shapes) != 0:
        face_ids = [face_recognition_model.compute_face_descriptor(im, shape, 1) for shape in shapes]
        face_ids_array = np.array(face_ids)
        face_ids_list = face_ids_array.tolist()[0]
        known_faces.append(face_ids_list)
    

    

    
print (len(known_faces))


6


# Running Application By Camera

In [6]:
import tensorflow as tf
import numpy as np
import dlib
import cv2


face_detector = dlib.get_frontal_face_detector()

shape_predictor = dlib.shape_predictor('Files/shape_predictor_68_face_landmarks.dat')

face_recognition_model = dlib.face_recognition_model_v1('Files/dlib_face_recognition_resnet_model_v1.dat')



def detect_face(im, sess, known_faces):
    images_test = []
    faces = face_detector(im, 1)
    shapes = [shape_predictor(im, face) for face in faces]
    face_ids = [face_recognition_model.compute_face_descriptor(im, shape, 1) for shape in shapes]
    face_ids_array = np.array(face_ids)
    
    in_test_set = True
    unknown_faces = []
    
    for i, face in enumerate(face_ids_array):
        for known_face in known_faces:
            thresh_hold = np.linalg.norm(known_face - face)
            if thresh_hold > 0.43:
                in_test_set = False
            else:
                in_test_set = True
                break
        
        if not in_test_set:
            font = cv2.FONT_HERSHEY_SIMPLEX
            cv2.putText(im,"Unknown Face",(faces[i].left()-50,faces[i].top()-50), font, 2,(0,255,0),2,cv2.LINE_AA)

            cv2.rectangle(im,(faces[i].left(),faces[i].top()),(faces[i].right(),faces[i].bottom()),(255,0,0),2)
            unknown_faces.append(i)
            

    face_ids_list = face_ids_array.tolist()
    
    if len(unknown_faces) != 0 and len(face_ids_list) != 0:
        for i, index in enumerate(unknown_faces):
            face_ids_list.pop(index - i)

    
    [images_test.append(face_id) for face_id in face_ids_list]
    
    if len(images_test) != 0:
        test_input = np.array(images_test)

        test_dict = {X: test_input, pkeep: 0.9}

        Y_predict = sess.run(Y, feed_dict=test_dict)

        for i, result in enumerate(Y_predict):
            confidence = round(result[np.argmax(result)] * 100, 2)
            number = np.argmax(result)
            predicted_face = dir_list[number] + str(confidence) + " %"
            font = cv2.FONT_HERSHEY_SIMPLEX
            cv2.putText(im,predicted_face,(faces[i].left()-50,faces[i].top()-50), font, 2,(0,255,0),2,cv2.LINE_AA)

            cv2.rectangle(im,(faces[i].left(),faces[i].top()),(faces[i].right(),faces[i].bottom()),(255,0,0),2)
    
    return im

cap = cv2.VideoCapture(1,cv2.CAP_DSHOW)

with tf.compat.v1.Session() as sess:
    saver.restore(sess, "Checkpoint/model.ckpt")
    print("Model restored.")

    while True:
        ret, frame = cap.read()
        cv2.imshow('Live stream', detect_face(frame, sess, known_faces))
        if cv2.waitKey(1) == 27: #esc to close
            break

cap.release()
cv2.destroyAllWindows() 

INFO:tensorflow:Restoring parameters from Checkpoint/model.ckpt
Model restored.


# Running Application By Picture

In [7]:
import tensorflow as tf
import numpy as np
import dlib
import cv2


face_detector = dlib.get_frontal_face_detector()

shape_predictor = dlib.shape_predictor('Files/shape_predictor_68_face_landmarks.dat')

face_recognition_model = dlib.face_recognition_model_v1('Files/dlib_face_recognition_resnet_model_v1.dat')

def detect_face_by_picture(data_path,name):
    im = mpimg.imread(data_path)
    if(file_name.endswith(".pgm")):
        im = np.stack((im,)*3, axis=-1)
    images_test = []
    faces = face_detector(im, 1)
    shapes = [shape_predictor(im, face) for face in faces]
    face_ids = [face_recognition_model.compute_face_descriptor(im, shape, 1) for shape in shapes]
    face_ids_array = np.array(face_ids)

    in_test_set = True
    unknown_faces = []
    indexNumber = 0
    for i, face in enumerate(face_ids_array):
        for known_face in known_faces:
            thresh_hold = np.linalg.norm(known_face - face)
            if thresh_hold > 0.45:
                in_test_set = False
            else:
                in_test_set = True
                print("knowface"+ name)
                return [1,dir_list[indexNumber]]
            indexNumber+=1

    if not in_test_set:
        print("Unknown Face:"+ name)
    return [0,"unkown"]

path = "Testing_Data/*/*"
total = 0
count = 0
for i, file_name in enumerate(glob.glob(path)):
    aa = list(file_name.split("\\"))[1]
    imageName = file_name.split("\\")[1]
    result = detect_face_by_picture(file_name,imageName)
    if(result[1] == aa):
        count += result[0]
    total += 1
print ("Accuracy: %s" % (count/total))

knowfaceChongyu
knowfaceChongyu
knowfaceChongyu
knowfaceDATA1
knowfaceDATA1
knowfaceDATA1
knowfaceDATA2
knowfaceDATA2
knowfaceDATA2
knowfaceJiehong
knowfaceJiehong
knowfaceJiehong
knowfaceKD
knowfaceKD
knowfaceKD
knowfaceKD
knowfaceKD
knowfaceKD
knowfaceKD
knowfaceKD
knowfaceKD
knowfaceKD
knowfaceKhoi
knowfaceKhoi
knowfaceKhoi
knowfaceKhoi
knowfaceKhoi
knowfaceKhoi
knowfaceKhoi
knowfaceKhoi
knowfaceKhoi
knowfaceKhoi
Accuracy: 0.8857142857142857
